In [1]:
import json
import requests
import citipy
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import gmaps
from config import gkey
cities_sample = pd.read_csv("../WeatherPy/cities_sample.csv")
cities_sample = cities_sample.rename(columns = {'Unnamed: 0': 'Country ID'})
gmaps.configure(api_key = gkey)

In [2]:
cities_sample.head()

,Country ID,Country,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,11663,FR,Hautmont,50.247704,3.924607,44.62,32.0,34.0,5.75
1,11875,FR,Nevers,46.991203,3.157084,57.20,76.0,94.0,9.17
2,43180,US,Payette,44.078333,-116.932778,50.09,40.0,75.0,8.05
3,34594,RO,Urechesti,46.133333,27.083333,50.00,76.0,75.0,6.93
4,722,AT,Scharnstein,47.900000,13.950000,55.00,89.0,100.0,4.00


In [3]:
locations = cities_sample[['Latitude', 'Longitude']]
max_humid = cities_sample['Humidity (%)'].max()

In [4]:
fig = gmaps.figure()
#Create a heat map that displays the humidity for every city from the part I of the homework.

humidity_layer = gmaps.heatmap_layer(locations, weights = cities_sample['Humidity (%)'], max_intensity = max_humid, point_radius = 10)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_temp = cities_sample.loc[((cities_sample['Temperature (F)'] < 80) & (cities_sample['Temperature (F)'] > 70))]
ideal_wind = ideal_temp.loc[cities_sample['Wind Speed (mph)'] < 10]
ideal_locations = ideal_wind.loc[ideal_wind['Cloudiness (%)'] == 0]
ideal_locations

,Country ID,Country,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
65,15034,HN,Morazan,15.316667,-87.6,74.16,67.0,0.0,2.39


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?key=" + gkey
hotel_df = pd.DataFrame([{'Name': "",
                        'City': "",
                        'Country': "",
                        'Latitude': "",
                        'Longitude': ""}], index = ideal_locations.index.values)
for index, row in ideal_locations.iterrows():
    lat, lng = row['Latitude'], row['Longitude']
    hotel_results = requests.get(base_url + f"&location={lat},{lng}&rankby=distance&keyword=hotel").json()['results']
    try:
        hotel_df.loc[index, 'Name'] = hotel_results[0]['name']
        hotel_df.loc[index, 'City'] = hotel_results[0]['vicinity']
        hotel_df.loc[index, 'Country'] = hotel_results[0]['plus_code']['compound_code'].split(',')[-1]
        hotel_df.loc[index, 'Latitude'] = hotel_results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Longitude'] = hotel_results[0]['geometry']['location']['lng']
    except:
        print(f"No hotels near {lat}, {lng}.")
hotel_df

,Name,City,Country,Latitude,Longitude
65,Hotel Diana's,Morazan,Honduras,15.3184,-87.6011


In [23]:
hotel_locations = [(row['Latitude'], row['Longitude']) for index, row in hotel_df.iterrows()]
info_box_template = """
<dt>Name:</dt> 
<dd>{Name}</dd>
<dt>City:</dt>
<dd>{City}</dd>
<dt>Country:</dt>
<dd>{Country}</dd>
"""
hotel_info = [{'Name': row['Name'],
              'City': row['City'],
              'Country': row['Country']} for index, row in hotel_df.iterrows()]

In [24]:
hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content = [info_box_template.format(**hotel) for hotel in hotel_info])
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))